CAPSTONE

Segmenting and Clustering Neighborhoods in New York City

In [5]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import urllib.request

In [6]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [7]:
class Scrapy:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(self.parse_html_table(table))\
                    for table in soup.find_all('table', class_="wikitable sortable")]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
            for row in table.find_all('tr'):
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        n_columns = len(td_tags)
                        
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [13]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
hp = Scrapy()
table = hp.parse_url(url)[0] 
table.head(15)

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


#Ignore cells with a borough that is "Not assigned"

In [14]:
table = table[table.Borough != 'Not assigned']
table.head(20)

,Postcode,Borough,Neighbourhood\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
10,M9A,Etobicoke,Islington Avenue\n
11,M1B,Scarborough,Rouge\n
12,M1B,Scarborough,Malvern\n


REMOVE '\n' from the data set

In [17]:
print(list(table))
table = table.replace('\n',' ', regex=True)
table.head(20)

['Postcode', 'Borough', 'Neighbourhood\n']


,Postcode,Borough,Neighbourhood\n
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [20]:
table = table[table['Neighbourhood\n'] != 'Not assigned']
table.head(20)

,Postcode,Borough,Neighbourhood\n
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


GROUPING BY NEIGHBORHOOD POSTAL CODE

In [24]:
df = table.groupby(['Postcode','Borough'])['Neighbourhood\n'].apply(lambda x: ", ".join(x.astype(str))).reset_index()
df = df.sample(frac=1).reset_index(drop=True)
df.head(20)

,Postcode,Borough,Neighbourhood\n
0,M1S,Scarborough,Agincourt
1,M3K,North York,"CFB Toronto , Downsview East"
2,M5R,Central Toronto,"The Annex , North Midtown , Yorkville"
3,M4A,North York,Victoria Village
4,M2N,North York,Willowdale South
5,M2H,North York,Hillcrest Village
6,M3N,North York,Downsview Northwest
7,M2J,North York,"Fairview , Henry Farm , Oriole"
8,M4R,Central Toronto,North Toronto West
9,M5V,Downtown Toronto,"CN Tower , Bathurst Quay , Island airport , Ha..."


In [26]:
print(df.shape)

(103, 3)


END OF PART 1

PART 2: GEOSPATIAL DATA

In [27]:
url2="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(url2)
geo_data.head(20)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [28]:
print(list(df))
print(list(geo_data))

full_table = df.set_index('Postcode').join(geo_data.set_index('Postal Code'))
full_table = full_table.sample(frac=1).reset_index(drop=True)
full_table.head(20)

['Postcode', 'Borough', 'Neighbourhood\n']
['Postal Code', 'Latitude', 'Longitude']


,Borough,Neighbourhood\n,Latitude,Longitude
0,North York,"Silver Hills , York Mills",43.757490,-79.374714
1,Downtown Toronto,"Design Exchange , Toronto Dominion Centre",43.647177,-79.381576
2,Downtown Toronto,Rosedale,43.679563,-79.377529
3,Scarborough,"Clairlea , Golden Mile , Oakridge",43.711112,-79.284577
4,East Toronto,"The Beaches West , India Bazaar",43.668999,-79.315572
5,East York,East Toronto,43.685347,-79.338106
6,Central Toronto,Davisville,43.704324,-79.388790
7,North York,"Bedford Park , Lawrence Manor East",43.733283,-79.419750
8,West Toronto,"Dovercourt Village , Dufferin",43.669005,-79.442259
9,Etobicoke,Northwest,43.706748,-79.594054


END OF PART 2